In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import json
import re
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [3]:
project_path = '/content/drive/MyDrive/Colab Notebooks/PatternMining'

# Load dataset

In [1]:
# file_path = f'{project_path}/datasets/all_video_games(cleaned).csv'
file_path = f'{project_path}/datasets/merged.csv'
data = pd.read_csv(file_path)

NameError: name 'project_path' is not defined

# PreProcess

In [89]:
# Set the allowable limit for missing values
allowable_limit = len(data) * 0.8  # You can adjust this threshold as per your preference

# Iterate over each column
for column in data.columns:
    # Count the missing values in each column
    missing_count = data[column].isnull().sum()

    # If missing values exceed the allowable limit, drop the column
    if missing_count > allowable_limit:
        data = data.drop(column, axis=1)
        print(f"Dropped column '{column}' due to exceeding allowable limit of missing values.")
    else:
        # For numerical columns, fill missing values with mean
        if data[column].dtype in ['int64', 'float64']:
            data.loc[:, column] = data[column].fillna(data[column].mean())
        else:
            data.loc[:, column] = data[column].fillna(data[column].mode()[0])


print("Missing values filled successfully.")
# print(data)


Dropped column 'jp_sales' due to exceeding allowable limit of missing values.
Missing values filled successfully.


# Manual preProcess

## Sales process

In [90]:
max_value = data['total_sales'].max()
min_value = data['total_sales'].min()
for column in ['total_sales', 'pal_sales', 'other_sales']:
    data[column] = (data[column] - min_value) / (max_value - min_value)
    # print(data[column])

In [91]:
data['other_sales'] = data['total_sales'] - data['pal_sales'] - data['na_sales']

## Platforms info process

In [92]:
def compute_max_metascore_info(informations):
    informations = informations.replace("'", '"')
    informations = json.loads(informations)
    max_metascore = 0
    max_metascore_count = 0
    for info in informations:
        pattern = r'\d+'

        metascore_count = 0
        metascore = 0

        if 'Platform Metascore Count' in info:
            match = re.search(pattern, info['Platform Metascore Count'])
            if match:
                metascore_count = int(match.group())

        if 'Platform Metascore' in info:
            match = re.search(pattern, info['Platform Metascore'])
            if match:
                metascore = int(match.group())

        if metascore_count > max_metascore_count:
            max_metascore_count = metascore_count
            max_metascore = metascore

    return max_metascore_count, max_metascore

In [93]:
# Apply the function to each row in the 'platforms info' column
max_metascore_info = data['platforms info'].apply(compute_max_metascore_info)

# Split the resulting tuples into separate columns
data[['max_metascore_count', 'max_metascore']] = pd.DataFrame(max_metascore_info.tolist(), index=data.index)

data.drop(columns=['platforms info'], inplace=True)
# Display the updated DataFrame
# print(data)

# Outlires

In [94]:
def remove_outliers(data, column):
  # Calculate the quantiles for Winsorization
  q1 = data[column].quantile(0.25)
  q3 = data[column].quantile(0.75)

  # Define the threshold for extreme values
  threshold = 1.5 * (q3 - q1)

  # Apply Winsorization
  data[column] = data[column].clip(lower=data[column].quantile(0.05), upper=data[column].quantile(0.95))
  return data

In [95]:
data = remove_outliers(data, 'na_sales')
data = remove_outliers(data, 'pal_sales')
data = remove_outliers(data, 'other_sales')
data = remove_outliers(data, 'max_metascore')
data = remove_outliers(data, 'max_metascore_count')

# Save dataset

In [96]:
data.to_csv(f'{project_path}/datasets/merged_cleaned.csv', index=False)

# Load Cleaned

In [4]:
data = pd.read_csv(f'{project_path}/datasets/merged_cleaned.csv')

In [5]:
data.drop(columns=['img', 'title', 'other_sales', 'release date_x', 'last_update', 'genres', 'genres splitted'], inplace=True)

In [6]:
# numeric_columns = ['critic_score', 'total_sales', 'na_sales', 'pal_sales', 'other_sales', 'user score', 'user ratings count', 'max_metascore_count', 'max_metascore']
numeric_columns = ['critic_score', 'total_sales', 'na_sales', 'pal_sales', 'user score', 'user ratings count', 'max_metascore_count', 'max_metascore']

for column in numeric_columns:
    data[column] = pd.cut(data[column], bins=3, labels=['low', 'medium', 'high'])

print(data.head())

  console   genre     publisher_x     developer_x critic_score total_sales  \
0     PS3  Action  Rockstar Games  Rockstar North         high        high   
1     PS4  Action  Rockstar Games  Rockstar North         high        high   
2    X360  Action  Rockstar Games  Rockstar North         high        high   
3    XOne  Action  Rockstar Games  Rockstar North         high      medium   
4      PC  Action  Rockstar Games  Rockstar North         high         low   

  na_sales pal_sales      product rating user score user ratings count  \
0     high      high  Rated M For Mature       high                low   
1     high      high  Rated M For Mature       high                low   
2     high      high  Rated M For Mature       high                low   
3     high      high  Rated M For Mature       high                low   
4   medium    medium  Rated M For Mature       high                low   

  max_metascore_count max_metascore  
0                high          high  
1         

In [7]:
# Convert categorical data to one-hot encoding
data_one_hot = pd.get_dummies(data)

# Display the one-hot encoded dataframe to check the changes
print(data_one_hot.head())


   console_2600  console_3DO  console_3DS  console_5200  console_7800  \
0         False        False        False         False         False   
1         False        False        False         False         False   
2         False        False        False         False         False   
3         False        False        False         False         False   
4         False        False        False         False         False   

   console_ACPC  console_AJ  console_All  console_Amig  console_And  ...  \
0         False       False        False         False        False  ...   
1         False       False        False         False        False  ...   
2         False       False        False         False        False  ...   
3         False       False        False         False        False  ...   
4         False       False        False         False        False  ...   

   user score_high  user ratings count_low  user ratings count_medium  \
0             True             

In [11]:
# Perform frequent pattern extraction using the apriori algorithm
min_support_threshold = 0.9
frequent_itemsets = apriori(data_one_hot, min_support=min_support_threshold, use_colnames=True)
print(frequent_itemsets)

    support                                           itemsets
0  0.917068                                (critic_score_high)
1  0.998121                                  (total_sales_low)
2  0.999952                           (user ratings count_low)
3  0.915237               (total_sales_low, critic_score_high)
4  0.917020        (user ratings count_low, critic_score_high)
5  0.998072          (total_sales_low, user ratings count_low)
6  0.915189  (total_sales_low, user ratings count_low, crit...


In [12]:
# Generate association rules with minimum confidence threshold
min_confidence_threshold = 0.9  # Adjust as per your dataset and requirements
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence_threshold)

# Display the rules
print(rules)

                                    antecedents  \
0                             (total_sales_low)   
1                           (critic_score_high)   
2                      (user ratings count_low)   
3                           (critic_score_high)   
4                             (total_sales_low)   
5                      (user ratings count_low)   
6     (user ratings count_low, total_sales_low)   
7          (total_sales_low, critic_score_high)   
8   (user ratings count_low, critic_score_high)   
9                             (total_sales_low)   
10                     (user ratings count_low)   
11                          (critic_score_high)   

                                    consequents  antecedent support  \
0                           (critic_score_high)            0.998121   
1                             (total_sales_low)            0.917068   
2                           (critic_score_high)            0.999952   
3                      (user ratings count_low)     